In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

In [2]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

dict是当日的最高价曾经达到过最高价

In [3]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

这一分钟内最高价，最低价，收盘价都与up_limit相同，判断为封板时刻

In [4]:
def up_first(stock):
    first = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[0][0]
            first.append(start_1)
        except:
            first.append(np.nan)
            pass
    return first
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first,dict.keys()),total = len(dict.keys())))
up_first_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [03:57<00:00, 18.63it/s]


In [5]:
ff.save('第一次封板时刻', up_first_factor)

第一次封板时刻成交量

In [6]:
def up_first_volume(stock):
    up_volume = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[0][0]
            volume = data.loc[:,'volume'][start_1]
            up_volume.append(volume)
        except:
            up_volume.append(np.nan)
            pass
    return up_volume
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first_volume,dict.keys()),total = len(dict.keys())))
up_first_volume_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_volume_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [03:56<00:00, 18.68it/s]


In [7]:
ff.save('第一次封板时刻成交量', up_first_volume_factor)

第一次封板时刻成交量占比

In [8]:
def up_first_volume_ratio(stock):
    volume_ratio = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[0][0]
            volume = data.loc[:,'volume'][start_1]
            volume_sum = data.loc[:,'volume'].sum()
            volume_ratio.append(volume/volume_sum)
        except:
            volume_ratio.append(np.nan)
            pass
    return volume_ratio
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first_volume_ratio,dict.keys()),total = len(dict.keys())))
up_first_volume_ratio_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_volume_ratio_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [04:19<00:00, 17.00it/s]


In [9]:
ff.save('第一次封板时刻成交量占比', up_first_volume_ratio_factor)

最后封板时刻

In [10]:
def up_last(stock):
    last = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[-1][0]
            try:
                last.append(start_1 - np.where(np.flip((judge).values[:start_1]) == False)[0][0])
            except:
                last.append(0)
        except:
            last.append(np.nan)
            pass
    return last
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_last,dict.keys()),total = len(dict.keys())))
up_last_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_last_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [04:46<00:00, 15.40it/s]


In [11]:
ff.save('最后封板时刻', up_last_factor)

最后封板时刻成交量

In [12]:
def up_last_volume(stock):
    up_volume = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[-1][0]
            try:
                last_1 = (start_1 - np.where(np.flip((judge).values[:start_1]) == False)[0][0])
            except:
                last_1 = 0
            volume = data.loc[:,'volume'][last_1]
            up_volume.append(volume)
        except:
            up_volume.append(np.nan)
            pass
    return up_volume
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_last_volume,dict.keys()),total = len(dict.keys())))
up_last_volume_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_last_volume_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [03:59<00:00, 18.43it/s]


In [13]:
ff.save('最后封板时刻成交量', up_last_volume_factor)

最后封板时刻成交量占比

In [14]:
def up_last_volume_ratio(stock):
    up_volume = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        volume_sum = data.loc[:,'volume'].sum()
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        try:
            start_1 = np.argwhere((judge).values)[-1][0]
            try:
                last_1 = (start_1 - np.where(np.flip((judge).values[:start_1]) == False)[0][0])
            except:
                last_1 = 0
            volume = data.loc[:,'volume'][last_1]
            up_volume.append(volume/volume_sum)
        except:
            up_volume.append(np.nan)
            pass
    return up_volume
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_last_volume_ratio,dict.keys()),total = len(dict.keys())))
up_last_volume_ratio_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_last_volume_ratio_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [04:28<00:00, 16.43it/s]


In [15]:
ff.save('最后封板时刻成交量占比', up_last_volume_ratio_factor)

板上成交量

In [16]:
def up_volume(stock):
    up_volume_list = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        volume_sum = data.loc[:,'volume'].sum()
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        volume = (judge * data.loc[:,'volume']).sum()
        if volume>0:
            up_volume_list.append(volume)
        else:
            up_volume_list.append(np.nan)
            pass
    return up_volume_list
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_volume,dict.keys()),total = len(dict.keys())))
up_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [05:18<00:00, 13.87it/s]


In [17]:
up_factor.count().sum()

51950

In [18]:
ff.save('板上成交量', up_factor)

板上成交量占比

In [19]:
def up_volume_ratio(stock):
    up_volume_list = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        volume_sum = data.loc[:,'volume'].sum()
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        volume = (judge * data.loc[:,'volume']).sum()
        if volume>0:
            up_volume_list.append(volume/volume_sum)
        else:
            up_volume_list.append(np.nan)
            pass
    return up_volume_list
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_volume_ratio,dict.keys()),total = len(dict.keys())))
up_ratio_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_ratio_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4417/4417 [04:53<00:00, 15.06it/s]


In [20]:
up_ratio_factor

,20200825,20200826,20200827,20200828,20200831,20200901,20200902,20200903,20200904,20200907,...,20240221,20240222,20240223,20240226,20240227,20240228,20240229,20240301,20240304,20240305
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.295367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004.SZ,NaN,NaN,NaN,NaN,0.178203,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.093085,NaN,NaN,NaN,NaN,NaN
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873703.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873706.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873726.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873806.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ff.save('板上成交量占比', up_ratio_factor)